In [1]:
import bs4 as bs
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import wraps
from bs4 import BeautifulSoup
import re
import requests
import time
import urllib.request
from typing import Union, List
import warnings

In [2]:
# URL & year
year = 2021
url = f"https://www.formula1.com/en/results/{year}/races"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html.parser")

## Data Parsing

#### Races year

In [5]:
# Saving Unique Races
seen_races = set()

all_races_df = []

# Possibility to change the year
race_pattern = rf"/en/results/{year}/races/(\d+)/([\w\-]+)/race-result"

for a in soup.find_all("a", href=True):
    match = re.search(race_pattern, a["href"])
    if match:
        race_id = int(match.group(1))
        race_name = match.group(2)

        if race_id in seen_races:
            continue
        seen_races.add(race_id)

        race_url = f"https://www.formula1.com/en/results/{year}/races/{race_id}/{race_name}/race-result"
        
        source = urllib.request.urlopen(race_url).read()
        race_soup = BeautifulSoup(source,'lxml')
        
        table = race_soup.find_all('table')[0]
        df_race = pd.read_html(str(table), flavor='bs4', header=[0])[0]
        
        df_race.insert(0, 'race_id', race_id)
        df_race.insert(1, 'race_name', race_name)

        all_races_df.append(df_race)
        time.sleep(0.5)

final_df = pd.concat(all_races_df, ignore_index=True)

/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/2599785373.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_race = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/2599785373.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_race = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/2599785373.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_race = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn

In [6]:
race = final_df.copy()

In [7]:
race.head()

,race_id,race_name,Pos.,No.,Driver,Team,Laps,Time / Retired,Pts.
0,1064,bahrain,1,44,Lewis HamiltonHAM,Mercedes,56,1:32:03.897,25.0
1,1064,bahrain,2,33,Max VerstappenVER,Red Bull Racing Honda,56,+0.745s,18.0
2,1064,bahrain,3,77,Valtteri BottasBOT,Mercedes,56,+37.383s,16.0
3,1064,bahrain,4,4,Lando NorrisNOR,McLaren Mercedes,56,+46.466s,12.0
4,1064,bahrain,5,11,Sergio PerezPER,Red Bull Racing Honda,56,+52.047s,10.0


In [8]:
race.to_csv('races_2021.csv')

#### Unique Practice

In [10]:
# Race pattern
race_pattern = rf"/en/results/{year}/races/(\d+)/([\w\-]+)/race-result"

# Unique races
seen_races = set()

# Empty lists for practice
all_practice = {1: [], 2: [], 3: []}

# go throught each link
for a in soup.find_all("a", href=True):
    match = re.search(race_pattern, a["href"])
    if not match:
        continue

    race_id = int(match.group(1))
    race_name = match.group(2)

    # rid of duplicates
    if race_id in seen_races:
        continue
    seen_races.add(race_id)

    # foe each session
    for session in [1, 2, 3]:

        practice_url = (
            f"https://www.formula1.com/en/results/{year}/races/"
            f"{race_id}/{race_name}/practice/{session}"
        )

        try:
            source = urllib.request.urlopen(practice_url).read()
        except:
            print("Page not found:", practice_url)
            continue

        practice_soup = BeautifulSoup(source, "lxml")
        tables = practice_soup.find_all("table")

        if not tables:
            print("No table found:", practice_url)
            continue

        df = pd.read_html(str(tables[0]), flavor="bs4", header=[0])[0]

        df.insert(0, "race_id", race_id)
        df.insert(1, "race_name", race_name)

        all_practice[session].append(df)

        time.sleep(0.5)

# df
practice_1_final = pd.concat(all_practice[1], ignore_index=True).drop_duplicates()
practice_2_final = pd.concat(all_practice[2], ignore_index=True).drop_duplicates()
practice_3_final = pd.concat(all_practice[3], ignore_index=True).drop_duplicates()

/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/3263397281.py:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[0]), flavor="bs4", header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/3263397281.py:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[0]), flavor="bs4", header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/3263397281.py:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[0]), flavor="bs4", header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/

In [11]:
practice_1 = practice_1_final.copy()
practice_2 = practice_2_final.copy()
practice_3 = practice_3_final.copy()

In [12]:
practice_1.head()

,race_id,race_name,Pos.,No.,Driver,Team,Time / Gap,Laps
0,1064,bahrain,1,33,Max VerstappenVER,Red Bull Racing Honda,1:31.394,12
1,1064,bahrain,2,77,Valtteri BottasBOT,Mercedes,+0.298s,17
2,1064,bahrain,3,4,Lando NorrisNOR,McLaren Mercedes,+0.503s,20
3,1064,bahrain,4,44,Lewis HamiltonHAM,Mercedes,+0.527s,15
4,1064,bahrain,5,16,Charles LeclercLEC,Ferrari,+0.599s,14


In [13]:
practice_2.head()

,race_id,race_name,Pos.,No.,Driver,Team,Time / Gap,Laps
0,1064,bahrain,1,33,Max VerstappenVER,Red Bull Racing Honda,1:30.847,23
1,1064,bahrain,2,4,Lando NorrisNOR,McLaren Mercedes,+0.095s,25
2,1064,bahrain,3,44,Lewis HamiltonHAM,Mercedes,+0.235s,24
3,1064,bahrain,4,55,Carlos SainzSAI,Ferrari,+0.280s,26
4,1064,bahrain,5,77,Valtteri BottasBOT,Mercedes,+0.371s,23


In [14]:
practice_3.head()

,race_id,race_name,Pos.,No.,Driver,Team,Time / Gap,Laps
0,1064,bahrain,1,33,Max VerstappenVER,Red Bull Racing Honda,1:30.577,11.0
1,1064,bahrain,2,44,Lewis HamiltonHAM,Mercedes,+0.739s,14.0
2,1064,bahrain,3,10,Pierre GaslyGAS,AlphaTauri Honda,+1.006s,13.0
3,1064,bahrain,4,77,Valtteri BottasBOT,Mercedes,+1.278s,16.0
4,1064,bahrain,5,11,Sergio PerezPER,Red Bull Racing Honda,+1.331s,11.0


In [15]:
practice_1.to_csv('practice_1_2021.csv')
practice_2.to_csv('practice_1_2021.csv')
practice_3.to_csv('practice_1_2021.csv')

#### Qualifying

In [17]:
qualification = []

for a in soup.find_all("a", href=True):
    match = re.search(rf"/en/results/{year}/races/(\d+)/([\w\-]+)/race-result", a["href"])
    if match:
        race_id = int(match.group(1))
        race_name = match.group(2)

        quali_url = f"https://www.formula1.com/en/results/{year}/races/{race_id}/{race_name}/qualifying"
        
        source = urllib.request.urlopen(quali_url).read()
        quali_soup = BeautifulSoup(source, 'lxml')
        
        tables = quali_soup.find_all('table')
        if not tables:
            print("No table found:", quali_url)
            continue

        table = tables[0]

        df_quali = pd.read_html(str(table), flavor='bs4', header=[0])[0]

        df_quali.insert(0, 'race_id', race_id)
        df_quali.insert(1, 'race_name', race_name)

        qualification.append(df_quali)
        time.sleep(.5)

qualifying_final = (
    pd.concat(qualification, ignore_index=True)
      .drop_duplicates()
      .reset_index(drop=True)
)

/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/3722388044.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_quali = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/3722388044.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_quali = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/3722388044.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_quali = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt0000

In [18]:
qualifying = qualifying_final.copy()

In [19]:
qualifying.head()

,race_id,race_name,Pos.,No.,Driver,Team,Q1,Q2,Q3,Laps
0,1064,bahrain,1,33,Max VerstappenVER,Red Bull Racing Honda,1:30.499,1:30.318,1:28.997,15
1,1064,bahrain,2,44,Lewis HamiltonHAM,Mercedes,1:30.617,1:30.085,1:29.385,18
2,1064,bahrain,3,77,Valtteri BottasBOT,Mercedes,1:31.200,1:30.186,1:29.586,17
3,1064,bahrain,4,16,Charles LeclercLEC,Ferrari,1:30.691,1:30.010,1:29.678,15
4,1064,bahrain,5,10,Pierre GaslyGAS,AlphaTauri Honda,1:30.848,1:30.513,1:29.809,15


In [20]:
qualifying.to_csv('qualifying_2021.csv')

#### Starting Grid

In [22]:
starting_position = []

for a in soup.find_all("a", href=True):
    match = re.search(rf"/en/results/{year}/races/(\d+)/([\w\-]+)/race-result", a["href"])
    if match:
        race_id = int(match.group(1))
        race_name = match.group(2)

        grid_url = f"https://www.formula1.com/en/results/{year}/races/{race_id}/{race_name}/starting-grid"
        
        source = urllib.request.urlopen(grid_url).read()
        grid_soup = BeautifulSoup(source, 'lxml')
        
        tables = grid_soup.find_all('table')
        if not tables:
            print("No table found:", grid_url)
            continue

        table = tables[0]

        df_grid = pd.read_html(str(table), flavor='bs4', header=[0])[0]

        df_grid.insert(0, 'race_id', race_id)
        df_grid.insert(1, 'race_name', race_name)

        starting_position.append(df_grid)
        time.sleep(.5)

starting_grid_final = (
    pd.concat(starting_position, ignore_index=True)
      .drop_duplicates()
      .reset_index(drop=True)
)

/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/3851791063.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_grid = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/3851791063.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_grid = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/3851791063.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_grid = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn

In [23]:
starting_grid = starting_grid_final.copy()

In [24]:
starting_grid.head()

,race_id,race_name,Pos.,No.,Driver,Team,Time
0,1064,bahrain,1,33,Max VerstappenVER,Red Bull Racing Honda,1:28.997
1,1064,bahrain,2,44,Lewis HamiltonHAM,Mercedes,1:29.385
2,1064,bahrain,3,77,Valtteri BottasBOT,Mercedes,1:29.586
3,1064,bahrain,4,16,Charles LeclercLEC,Ferrari,1:29.678
4,1064,bahrain,5,10,Pierre GaslyGAS,AlphaTauri Honda,1:29.809


In [25]:
starting_grid.to_csv('starting_grid_2021.csv')

#### Pit Stops

In [27]:
pit_stop_summary = []

for a in soup.find_all("a", href=True):
    match = re.search(rf"/en/results/{year}/races/(\d+)/([\w\-]+)/race-result", a["href"])
    if match:
        race_id = int(match.group(1))
        race_name = match.group(2)

        pit_url = f"https://www.formula1.com/en/results/{year}/races/{race_id}/{race_name}/pit-stop-summary"
        
        source = urllib.request.urlopen(pit_url).read()
        pit_soup = BeautifulSoup(source, 'lxml')
        
        tables = pit_soup.find_all('table')
        if not tables:
            print("No table found:", pit_url)
            continue

        table = tables[0]

        df_pit = pd.read_html(str(table), flavor='bs4', header=[0])[0]

        df_pit.insert(0, 'race_id', race_id)
        df_pit.insert(1, 'race_name', race_name)

        pit_stop_summary.append(df_pit)
        time.sleep(.5)

pit_stop_final = (
    pd.concat(pit_stop_summary, ignore_index=True)
      .drop_duplicates()
      .reset_index(drop=True)
)

/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/2525141044.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_pit = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/2525141044.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_pit = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/2525141044.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_pit = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/

In [28]:
pit_stop = pit_stop_final.copy()

In [29]:
pit_stop.head()

,race_id,race_name,Stops,No.,Driver,Team,Lap,Time of Day,Time,Total
0,1064,bahrain,1,11,Sergio PerezPER,Red Bull Racing Honda,2,18:11:56,23.993,23.993
1,1064,bahrain,1,10,Pierre GaslyGAS,AlphaTauri Honda,4,18:16:14,38.338,38.338
2,1064,bahrain,1,14,Fernando AlonsoALO,Alpine Renault,11,18:27:32,24.373,24.373
3,1064,bahrain,1,4,Lando NorrisNOR,McLaren Mercedes,12,18:29:05,24.899,24.899
4,1064,bahrain,1,16,Charles LeclercLEC,Ferrari,12,18:29:06,24.925,24.925


In [30]:
pit_stop.to_csv('pit_stop_2021.csv')

#### Fastest lap

In [32]:
fastest_lap_summary = []

for a in soup.find_all("a", href=True):
    match = re.search(rf"/en/results/{year}/races/(\d+)/([\w\-]+)/race-result", a["href"])
    if match:
        race_id = int(match.group(1))
        race_name = match.group(2)

        fastest_url = f"https://www.formula1.com/en/results/{year}/races/{race_id}/{race_name}/fastest-laps"
        
        source = urllib.request.urlopen(fastest_url).read()
        fastest_soup = BeautifulSoup(source, 'lxml')
        
        tables = fastest_soup.find_all('table')
        if not tables:
            print("No table found:", fastest_url)
            continue

        table = tables[0]

        df_fastest = pd.read_html(str(table), flavor='bs4', header=[0])[0]
        df_fastest.insert(0, 'race_id', race_id)
        df_fastest.insert(1, 'race_name', race_name)

        fastest_lap_summary.append(df_fastest)
        time.sleep(.5)

fastest_lap_final = (
    pd.concat(fastest_lap_summary, ignore_index=True)
      .drop_duplicates()
      .reset_index(drop=True)
)

/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/2838962706.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_fastest = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/2838962706.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_fastest = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/2838962706.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_fastest = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pc

In [33]:
fastest_lap = fastest_lap_final.copy()

In [34]:
fastest_lap.head()

,race_id,race_name,Pos.,No.,Driver,Team,Lap,Time of Day,Time,Avg. Speed
0,1064,bahrain,1,77,Valtteri BottasBOT,Mercedes,56,19:39:51,1:32.090,211.566
1,1064,bahrain,2,33,Max VerstappenVER,Red Bull Racing Honda,41,19:15:41,1:33.228,208.984
2,1064,bahrain,3,11,Sergio PerezPER,Red Bull Racing Honda,44,19:21:10,1:33.970,207.334
3,1064,bahrain,4,44,Lewis HamiltonHAM,Mercedes,44,19:20:17,1:34.015,207.235
4,1064,bahrain,5,10,Pierre GaslyGAS,AlphaTauri Honda,48,19:28:30,1:34.090,207.069


In [35]:
fastest_lap.to_csv('fastest_lap_2021.csv')

#### Sprint Grid

In [37]:
sprint_grid_summary = []

for a in soup.find_all("a", href=True):
    match = re.search(rf"/en/results/{year}/races/(\d+)/([\w\-]+)/race-result", a["href"])
    if match:
        race_id = int(match.group(1))
        race_name = match.group(2)

        sprint_grid_url = f"https://www.formula1.com/en/results/{year}/races/{race_id}/{race_name}/sprint-grid"
        
        source = urllib.request.urlopen(sprint_grid_url).read()
        sprint_grid_soup = BeautifulSoup(source, 'lxml')
        
        tables = sprint_grid_soup.find_all('table')
        if not tables:
            print("No table found:", sprint_grid_url)
            continue

        table = tables[0]

        df_sprint_grid = pd.read_html(str(table), flavor='bs4', header=[0])[0]
        df_sprint_grid.insert(0, 'race_id', race_id)
        df_sprint_grid.insert(1, 'race_name', race_name)

        sprint_grid_summary.append(df_sprint_grid)
        time.sleep(.5)

sprint_grid_final = (
    pd.concat(sprint_grid_summary, ignore_index=True)
      .drop_duplicates()
      .reset_index(drop=True)
)

/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/3667383370.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_sprint_grid = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/3667383370.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_sprint_grid = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/3667383370.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_sprint_grid = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h1053

In [38]:
sprint_grid = sprint_grid_final.copy()

In [39]:
sprint_grid.head()

,race_id,race_name,Pos.,No.,Driver,Team,Time
0,1064,bahrain,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available
1,1065,emilia-romagna,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available
2,1066,portugal,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available
3,1086,spain,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available
4,1067,monaco,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available


In [40]:
sprint_grid.to_csv('sprint_grid_2021.csv')

#### Sprint

In [42]:
sprint_summary = []

for a in soup.find_all("a", href=True):
    match = re.search(rf"/en/results/{year}/races/(\d+)/([\w\-]+)/race-result", a["href"])
    if match:
        race_id = int(match.group(1))
        race_name = match.group(2)

        sprint_url = f"https://www.formula1.com/en/results/{year}/races/{race_id}/{race_name}/sprint-results"
        
        source = urllib.request.urlopen(sprint_url).read()
        sprint_soup = BeautifulSoup(source, 'lxml')
        
        tables = sprint_soup.find_all('table')
        if not tables:
            print("No table found:", sprint_url)
            continue

        table = tables[0]

        df_sprint = pd.read_html(str(table), flavor='bs4', header=[0])[0]
        df_sprint.insert(0, 'race_id', race_id)
        df_sprint.insert(1, 'race_name', race_name)

        sprint_summary.append(df_sprint)
        time.sleep(.5)

sprint_final = (
    pd.concat(sprint_summary, ignore_index=True)
      .drop_duplicates()
      .reset_index(drop=True)
)

/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/3573704070.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_sprint = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/3573704070.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_sprint = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/3573704070.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_sprint = pd.read_html(str(table), flavor='bs4', header=[0])[0]
/var/folders/f0/nc46h10539q_2c9xxx6pctt0

In [43]:
sprint = sprint_final.copy()

In [44]:
sprint.head()

,race_id,race_name,Pos.,No.,Driver,Team,Laps,Time / Retired,Pts.
0,1064,bahrain,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available
1,1065,emilia-romagna,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available
2,1066,portugal,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available
3,1086,spain,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available
4,1067,monaco,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available,ErrorNo results available


In [45]:
sprint.to_csv('sprint_2021.csv')

## Team Standings

In [47]:
url_fastest_car = f'https://www.formula1.com/en/results/{year}/team'
resp = requests.get(url_fastest_car)
soup = BeautifulSoup(resp.text, "html.parser")

In [48]:
source = urllib.request.urlopen(url_fastest_car).read()
soup = BeautifulSoup(source, 'lxml')

table = soup.find_all('table')[0]

team_standings = pd.read_html(str(table), flavor='bs4', header=0)[0]

/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/4035238601.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team_standings = pd.read_html(str(table), flavor='bs4', header=0)[0]


In [49]:
team_positions = team_standings.copy()

In [61]:
team_positions = team_positions.rename(columns={'Pos.':'Position', 'Pts.':'Points'})

In [63]:
team_positions.head()

,Position,Team,Points
0,1,Mercedes,613.5
1,2,Red Bull Racing Honda,585.5
2,3,Ferrari,323.5
3,4,McLaren Mercedes,275.0
4,5,Alpine Renault,155.0


In [65]:
team_positions.to_csv('team_positions_2021.csv')

## Winner

In [67]:
url_winner = f'https://www.formula1.com/en/results/{year}/drivers'
resp = requests.get(url_winner)
soup = BeautifulSoup(resp.text, "html.parser")

In [69]:
source = urllib.request.urlopen(url_winner).read()
soup = BeautifulSoup(source, 'lxml')

table = soup.find_all('table')[0]

drivers_standings = pd.read_html(str(table), flavor='bs4', header=0)[0]

/var/folders/f0/nc46h10539q_2c9xxx6pctt00000gn/T/ipykernel_21665/3398591731.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  drivers_standings = pd.read_html(str(table), flavor='bs4', header=0)[0]


In [71]:
standings = drivers_standings.copy()

In [73]:
standings.head()

,Pos.,Driver,Nationality,Team,Pts.
0,1,Max VerstappenVER,NED,Red Bull Racing Honda,395.5
1,2,Lewis HamiltonHAM,GBR,Mercedes,387.5
2,3,Valtteri BottasBOT,FIN,Mercedes,226.0
3,4,Sergio PerezPER,MEX,Red Bull Racing Honda,190.0
4,5,Carlos SainzSAI,ESP,Ferrari,164.5


In [77]:
standings = standings.rename(columns={'Pos.':'Position', 'Pts.':'Points'})

In [79]:
standings.head()

,Position,Driver,Nationality,Team,Points
0,1,Max VerstappenVER,NED,Red Bull Racing Honda,395.5
1,2,Lewis HamiltonHAM,GBR,Mercedes,387.5
2,3,Valtteri BottasBOT,FIN,Mercedes,226.0
3,4,Sergio PerezPER,MEX,Red Bull Racing Honda,190.0
4,5,Carlos SainzSAI,ESP,Ferrari,164.5


In [81]:
standings.to_csv('standings_2021.csv')